In [1]:
import os
import faiss
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

## Этап 1. Загрузка текстов из файлов

In [2]:
TEXT_FILES_DIR = "garant"

In [3]:
def load_texts_from_folder(folder_path):
    texts = []
    filenames = []
    for file in tqdm(sorted(os.listdir(folder_path)), desc="Загрузка текстов"):
        if file.endswith(".txt"):
            file_path = os.path.join(folder_path, file)
            with open(file_path, "r", encoding="utf-8") as f:
                text = f.read().strip()
                texts.append(text)
                filenames.append(file)
    return texts, filenames

In [4]:
texts, filenames = load_texts_from_folder(TEXT_FILES_DIR)

Загрузка текстов: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 1628/1628 [00:19<00:00, 82.07it/s]


## Этап 2. Создание эмбеддингов

In [5]:
MODEL_NAME = "Sberbank-ai/sbert_large_mt_nlu_ru"
model = SentenceTransformer(MODEL_NAME)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/866 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [6]:
text_embeddings = model.encode(texts, batch_size=16, convert_to_numpy=True, show_progress_bar=True)
print(f"\nРазмерность эмбеддингов: {text_embeddings.shape}")

Batches:   0%|          | 0/102 [00:00<?, ?it/s]


Размерность эмбеддингов: (1628, 1024)


## Этап 3: Создание FAISS-индекса

FAISS используется для быстрого поиска среди эмбеддингов

In [7]:
norms = np.linalg.norm(text_embeddings, axis=1, keepdims=True)
text_embeddings_normalized = text_embeddings / norms
dimension = text_embeddings.shape[1]

Создаём FAISS-индекс

In [8]:
index = faiss.IndexFlatL2(dimension)
index.add(text_embeddings_normalized.astype('float32'))

Сохраняем индекс для последующего использования

In [9]:
faiss.write_index(index, "results_SBER_Default/text_index.faiss")
np.save("results_SBER_Default/filenames.npy", np.array(filenames))  # сохраняем список имён файлов

## Этап 4: Функция поиска похожих текстов

In [10]:
def find_similar_texts(query, top_k=5):
    query_embedding = model.encode([query], convert_to_numpy=True) # преобразуем запрос в эмбеддинг
    distances, indices = index.search(query_embedding, top_k) # выполняем поиск в FAISS

    print("Наиболее похожие тексты:")
    for i, idx in enumerate(indices[0]):
        print(f"{i+1}. {filenames[idx]} (дистанция: {distances[0][i]:.4f})")
        print(texts[idx][:300] + "...")  # выводим первые 300 символов текста
        print("-" * 80)

In [11]:
query_text = "Международный день инвалидов в Хабаровске"
find_similar_texts(query_text, top_k=20)

Наиболее похожие тексты:
1. garant_1261.txt (дистанция: 0.8277)
﻿
Постановление Мэра г. Хабаровска
от 11 ноября 2004 г. N 1650
"О проведении Международного дня инвалидов в городе Хабаровске в 2004 году"

В связи с проведением 3 декабря 2004 года Международного дня инвалидов и в целях привлечения внимания широких слоев общественности города, предприятий, учрежден...
--------------------------------------------------------------------------------
2. garant_1223.txt (дистанция: 0.9793)
﻿
Постановление главы администрации Иркутской области от 24 сентября 2004 г. N 179-ПГ
"О признании утратившими силу п.п. 3.5, 3.6 Порядка возмещения расходов на реализацию
отдельных льгот, установленных Федеральным законом "О ветеранах",
утвержденного постановлением губернатора области
от 13.04.2004...
--------------------------------------------------------------------------------
3. garant_0198.txt (дистанция: 1.0475)
﻿
Закон Калининградской области от 30 декабря 2004 г. N 483
"О внесении изменений в Зако